<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto, Canada</font></h1>

## Part 1 Assignment Tasks
1. Start by creating a new Notebook for this assignment. -Done

3. To create the above dataframe:

    * The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    * Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    * More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.
    * If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
    * Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    * In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
4. Submit a link to your Notebook on <b>your Github repository. (10 marks)</b>

<b>Note:</b> There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.



In [18]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import wget #to import csv file that has latitude and longitude of a location
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [52]:
#getting html text from the Wikipedia page url given in assignment
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
url[0:500]

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"8b40cdc2-a77e-4bb3-8d5'

## Parsing the wikipedia webpage using BeautifulSoup package
Beautiful Soup is a Python package for parsing HTML and XML documents. It creates a parse tree for parsed pages that can be used to extract data from HTML, which is useful for web scraping. 
### Prettify() function
Prettify() function in BeautifulSoup will enable us to view how the tags are nested in the document.

In [53]:
# parse HTML using BeautifulSoup
from bs4 import BeautifulSoup
soup = BeautifulSoup(url,'lxml')
print(soup.prettify()[0:500])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"8b40


### Extracting the table
The table is in the loop "wikitable sortable". find the table and extract the table content
   

In [56]:
zipcode_table_from_text = soup.find('table',{'class':'wikitable sortable'})

zipcode_table_from_text.text[0:500]

"\n\nPostal Code\n\nBorough\n\nNeighbourhood\n\n\nM1A\n\nNot assigned\n\nNot assigned\n\n\nM2A\n\nNot assigned\n\nNot assigned\n\n\nM3A\n\nNorth York\n\nParkwoods\n\n\nM4A\n\nNorth York\n\nVictoria Village\n\n\nM5A\n\nDowntown Toronto\n\nRegent Park, Harbourfront\n\n\nM6A\n\nNorth York\n\nLawrence Manor, Lawrence Heights\n\n\nM7A\n\nDowntown Toronto\n\nQueen's Park, Ontario Provincial Government\n\n\nM8A\n\nNot assigned\n\nNot assigned\n\n\nM9A\n\nEtobicoke\n\nIslington Avenue, Humber Valley Village\n\n\nM1B\n\nScarborough\n\nMalvern, Rouge\n\n\nM2B\n\nNot assigned\n\nNot assig"

### break the table with tr rows
Break the text to the rows to iterate and extract text

In [60]:
table_data = zipcode_table_from_text.findAll('tr')

table_data[0:2]

[<tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighbourhood
 </th></tr>,
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>]

In [6]:
# extract the column names from first row
column_names = [th.text.replace("\n", "") for th in table_data[0].findAll('th')]
data_table = []

column_names

['Postal Code', 'Borough', 'Neighbourhood']

### Iterate each tr row
Each tr row consists of td row that contains 'Postal Code', 'Borough', 'Neighbourhood' data

In [7]:
for tr in table_data:
    tmp = [td.text.replace("\n","") for td in tr.findAll('td')]
    if(len(tmp) > 0): data_table.append(tmp)

zip_df = pd.DataFrame(data_table, columns=column_names)
print('zip_df DataFrame size:', zip_df.shape)
zip_df

zip_df DataFrame size: (180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


### remove 'Not Assigned' Neighbourhoods from the dataframe

In [8]:
zip_df = zip_df[zip_df['Neighbourhood'] != 'Not assigned'].reset_index(drop=True)
zip_df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [9]:
print('zip_df DataFrame size:', zip_df.shape)

zip_df DataFrame size: (103, 3)


### Part 1 Assignment FootNotes:
I have split Neighborhoods with commas "," in Part3 of this assignment below as it was not requested specifically as a task for Part 1

# Part2 Assignment Tasks

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code.

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

<b>Use the Geocoder package or the csv file to create the dataframe: 'Postal Code', 'Burough', 'Neighborhood', 'Latitude', 'Longitude'</b>

Important Note: There is a limit on how many times you can call geocoder.google function. It is 2500 times per day. This should be way more than enough for you to get acquainted with the package and to use it to get the geographical coordinates of the neighborhoods in the Toronto.

<b>Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. (2 marks)</b>

### Geocoder package did not work, so using csv file

In [ ]:
#Download the csv file
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

In [10]:
#read the csv file into DataFrame
lat_long_df = pd.read_csv('Geospatial_Coordinates.csv')
lat_long_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
#create new DataFrame with co-ordinates data and neighborhoods dataframe by merging them on postal code column
city_df = pd.merge(zip_df, lat_long_df, on = "Postal Code", how = "inner")
city_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [12]:
#correcting Neighbourhood column name
city_df.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True) 
city_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [13]:
city_df.shape

(103, 5)

# Part 3 Assignment Tasks

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
to generate maps to visualize your neighborhoods and how they cluster together. 
Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)


####  I want to first see how spread out the entire dataset is on the Map using Folium library

In [14]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(city_df['Borough'].unique()),
        city_df.shape[0]
    )
)
print('The different buroughs are',city_df['Borough'].unique())

The dataframe has 10 boroughs and 103 neighborhoods.
The different buroughs are ['North York' 'Downtown Toronto' 'Etobicoke' 'Scarborough' 'East York'
 'York' 'East Toronto' 'West Toronto' 'Central Toronto' 'Mississauga']


#### Splitting multiple Neighborhoods with comma "," in Neighborhood column

In [15]:
column_names = city_df.columns
data_table=[]
for index,row in city_df.iterrows():
    if("," in row['Neighborhood']):
        for nh in row['Neighborhood'].split(','):
            data_table.append([row["Postal Code"],row["Borough"], nh.strip(), row["Latitude"],row["Longitude"]])
    else:
        data_table.append([row["Postal Code"],row["Borough"], row['Neighborhood'], row["Latitude"],row["Longitude"]])
city_df = pd.DataFrame(data_table, columns = column_names)
city_df  

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
...,...,...,...,...,...
212,M8Z,Etobicoke,Mimico NW,43.628841,-79.520999
213,M8Z,Etobicoke,The Queensway West,43.628841,-79.520999
214,M8Z,Etobicoke,South of Bloor,43.628841,-79.520999
215,M8Z,Etobicoke,Kingsway Park South West,43.628841,-79.520999


### Checking Toronto in Burough data
I want to check data with only boroughs that contain the word Toronto

In [21]:
toronto_df = city_df[city_df.Borough.isin(['Downtown Toronto' , 'East Toronto', 'West Toronto' ,'Central Toronto' ])]
print('The size of DataFrame is ',toronto_df.shape)
toronto_df.head()


The size of DataFrame is  (78, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
6,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
7,M7A,Downtown Toronto,Ontario Provincial Government,43.662301,-79.389494
15,M5B,Downtown Toronto,Garden District,43.657162,-79.378937


Mapping the Toronto only Burough map data

In [23]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [24]:

# create map of Toronto burough using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version

In [25]:
# @hidden_cell
CLIENT_ID = 'VWPKXD3BHZ4JHGDMHBYNR2NHFFRIDCWCSMIWZGER0EIRTOJF' # your Foursquare ID
CLIENT_SECRET = 'TIM15AK24FT0ZWBJCWAZ3RUPP1BTXQFFWOIWI5GLMUZWSTS2' # your Foursquare Secret
ACCESS_TOKEN = 'HWDJK45NWTWWSCO1VYJUAQMEQE53BVM01EATGE0GLA2TXOU3' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [26]:
# type your answer here
LIMIT = 100
RADIUS = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude,
    VERSION, 
    20, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VWPKXD3BHZ4JHGDMHBYNR2NHFFRIDCWCSMIWZGER0EIRTOJF&client_secret=TIM15AK24FT0ZWBJCWAZ3RUPP1BTXQFFWOIWI5GLMUZWSTS2&ll=43.6534817,-79.3839347&v=20180605&radius=20&limit=100'

## Explore neighborhoods in toronto_df using Foursquare
create a function to explore all the neighborhoods in toronto_df

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        #print(url)    
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
#dataframe to store venue data for each neighborhood, also shows list fo neighborhoods explored by the function call
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Regent Park
Harbourfront
Queen's Park
Ontario Provincial Government
Garden District
Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond
Adelaide
King
Dufferin
Dovercourt Village
Harbourfront East
Union Station
Toronto Islands
Little Portugal
Trinity
The Danforth West
Riverdale
Toronto Dominion Centre
Design Exchange
Brockton
Parkdale Village
Exhibition Place
India Bazaar
The Beaches West
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
Forest Hill Road Park
High Park
The Junction South
North Toronto West
Lawrence Park
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
University of Toronto
Harbord
Runnymede
Swansea
Moore Park
Summerhill East
Kensington Market
Chinatown
Grange Park
Summerhill West
Rathnelly
South Hill
Forest Hill SE
Deer Park
CN Tower
King and Spadina
Railway Lands
Harbourfront West
Bathurst Quay
South Niagara
Island airport
Rosedale
Stn A PO Boxes
St. James Tow

#### Let's check the size of the resulting dataframe

In [29]:
print(toronto_venues.shape)
toronto_venues.head()

(3174, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Regent Park,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Let's check how many venues were returned for each neighborhood

In [30]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,95,95,95,95,95,95
Bathurst Quay,17,17,17,17,17,17
Berczy Park,61,61,61,61,61,61
Brockton,22,22,22,22,22,22
Business reply mail Processing Centre,16,16,16,16,16,16
...,...,...,...,...,...,...
Underground city,100,100,100,100,100,100
Union Station,100,100,100,100,100,100
University of Toronto,34,34,34,34,34,34


#### Let's find out how many unique categories can be curated from all the returned venues

In [31]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 232 uniques categories.


### Analyze each neighborhood

In [32]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
print('The size of the toronto_onehot dataframe is ',toronto_onehot.shape)

The size of the toronto_onehot dataframe is  (3174, 232)


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [34]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Adelaide,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021053,...,0.010526,0.0,0.0,0.0,0.0,0.00,0.010526,0.000000,0.0,0.00
1,Bathurst Quay,0.000000,0.0,0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00
2,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.016393,0.000000,0.0,0.00
3,Brockton,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00
4,Business reply mail Processing Centre,0.062500,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,Underground city,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.010000,0.0,0.0,0.0,0.0,0.01,0.010000,0.000000,0.0,0.01
72,Union Station,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.0,0.0,0.0,0.0,0.01,0.010000,0.000000,0.0,0.01
73,University of Toronto,0.029412,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.029412,0.0,0.0,0.0,0.0,0.00,0.000000,0.029412,0.0,0.00
74,Victoria Hotel,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.020000,0.000000,0.0,0.01


In [35]:
print('the size of toronto_grouped is ',toronto_grouped.shape)

the size of toronto_grouped is  (76, 232)


#### 5 most common venues of each neighborhood

In [36]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
             venue  freq
0      Coffee Shop  0.09
1             Café  0.05
2       Restaurant  0.04
3    Deli / Bodega  0.03
4  Thai Restaurant  0.03


----Bathurst Quay----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3       Coffee Shop  0.06
4             Plane  0.06


----Berczy Park----
          venue  freq
0   Coffee Shop  0.10
1        Bakery  0.05
2  Cocktail Bar  0.05
3    Restaurant  0.03
4      Pharmacy  0.03


----Brockton----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3    Intersection  0.05
4   Burrito Place  0.05


----Business reply mail Processing Centre----
                  venue  freq
0           Yoga Studio  0.06
1         Auto Workshop  0.06
2  Gym / Fitness Center  0.06
3         Garden Center  0.06
4                Garden  0.06


----CN Tower----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport 

         venue  freq
0  Coffee Shop  0.10
1         Café  0.07
2        Hotel  0.05
3          Gym  0.04
4   Restaurant  0.04


----Union Station----
            venue  freq
0     Coffee Shop  0.13
1        Aquarium  0.05
2            Café  0.04
3           Hotel  0.04
4  Scenic Lookout  0.03


----University of Toronto----
                 venue  freq
0                 Café  0.15
1               Bakery  0.06
2  Japanese Restaurant  0.06
3            Bookstore  0.06
4                  Bar  0.06


----Victoria Hotel----
                venue  freq
0         Coffee Shop  0.13
1          Restaurant  0.07
2                Café  0.06
3               Hotel  0.05
4  Italian Restaurant  0.04


----Yorkville----
               venue  freq
0     Sandwich Place  0.15
1               Café  0.15
2        Coffee Shop  0.10
3               Park  0.05
4  Indian Restaurant  0.05




Let's save the data into dataframe

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

 let's create the new dataframe and display the top 10 venues for each neighborhood.

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Restaurant,Gym,Clothing Store,Thai Restaurant,Deli / Bodega,Sushi Restaurant,Bookstore,Hotel
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boutique,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Harbor / Marina
2,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Beer Bar,Restaurant,Pharmacy,Farmers Market,Cheese Shop,Museum
3,Brockton,Café,Coffee Shop,Breakfast Spot,Bakery,Convenience Store,Performing Arts Venue,Pet Store,Climbing Gym,Restaurant,Burrito Place
4,Business reply mail Processing Centre,Yoga Studio,Auto Workshop,Gym / Fitness Center,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Pizza Place


## Cluster Neighborhoods
run a k-means cluster into 11 clusters

In [40]:
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

In [41]:

# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 1, 1, 1, 4, 1, 1, 1, 1])

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [44]:

toronto_merged = toronto_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,1,Coffee Shop,Pub,Park,Café,Bakery,Theater,Breakfast Spot,Brewery,Spa,Beer Store
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1,Coffee Shop,Pub,Park,Café,Bakery,Theater,Breakfast Spot,Brewery,Spa,Beer Store
6,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,College Cafeteria,Diner,Bar,Smoothie Shop,Beer Bar,Japanese Restaurant,Sandwich Place,Distribution Center
7,M7A,Downtown Toronto,Ontario Provincial Government,43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,College Cafeteria,Diner,Bar,Smoothie Shop,Beer Bar,Japanese Restaurant,Sandwich Place,Distribution Center
15,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,1,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Lingerie Store,Japanese Restaurant,Hotel,Ramen Restaurant


let's visualize the clusters

In [45]:
import matplotlib.cm as cm
import matplotlib.colors as colors


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters
examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

#### Cluster1

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
162,Central Toronto,0,Coffee Shop,Sushi Restaurant,American Restaurant,Supermarket,Pub,Bank,Fried Chicken Joint,Pizza Place,Restaurant,Bagel Shop
163,Central Toronto,0,Coffee Shop,Sushi Restaurant,American Restaurant,Supermarket,Pub,Bank,Fried Chicken Joint,Pizza Place,Restaurant,Bagel Shop
164,Central Toronto,0,Coffee Shop,Sushi Restaurant,American Restaurant,Supermarket,Pub,Bank,Fried Chicken Joint,Pizza Place,Restaurant,Bagel Shop
165,Central Toronto,0,Coffee Shop,Sushi Restaurant,American Restaurant,Supermarket,Pub,Bank,Fried Chicken Joint,Pizza Place,Restaurant,Bagel Shop
166,Central Toronto,0,Coffee Shop,Sushi Restaurant,American Restaurant,Supermarket,Pub,Bank,Fried Chicken Joint,Pizza Place,Restaurant,Bagel Shop


#### Cluster2

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Coffee Shop,Pub,Park,Café,Bakery,Theater,Breakfast Spot,Brewery,Spa,Beer Store
3,Downtown Toronto,1,Coffee Shop,Pub,Park,Café,Bakery,Theater,Breakfast Spot,Brewery,Spa,Beer Store
6,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,College Cafeteria,Diner,Bar,Smoothie Shop,Beer Bar,Japanese Restaurant,Sandwich Place,Distribution Center
7,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,College Cafeteria,Diner,Bar,Smoothie Shop,Beer Bar,Japanese Restaurant,Sandwich Place,Distribution Center
15,Downtown Toronto,1,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Lingerie Store,Japanese Restaurant,Hotel,Ramen Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
196,Downtown Toronto,1,Coffee Shop,Café,Hotel,Japanese Restaurant,Gym,Restaurant,Salad Place,American Restaurant,Steakhouse,Seafood Restaurant
197,Downtown Toronto,1,Coffee Shop,Café,Hotel,Japanese Restaurant,Gym,Restaurant,Salad Place,American Restaurant,Steakhouse,Seafood Restaurant
201,Downtown Toronto,1,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Restaurant,Gay Bar,Yoga Studio,Hotel,Mediterranean Restaurant,Men's Store,Dance Studio
202,East Toronto,1,Yoga Studio,Auto Workshop,Gym / Fitness Center,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Pizza Place


#### Cluster3

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
114,Central Toronto,2,Garden,Wine Bar,Dance Studio,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


#### Cluster4

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
123,Central Toronto,3,Park,Jewelry Store,Trail,Sushi Restaurant,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
124,Central Toronto,3,Park,Jewelry Store,Trail,Sushi Restaurant,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
187,Downtown Toronto,3,Park,Trail,Playground,Cuban Restaurant,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


#### Cluster5

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
167,Downtown Toronto,4,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boutique,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Harbor / Marina
168,Downtown Toronto,4,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boutique,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Harbor / Marina
169,Downtown Toronto,4,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boutique,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Harbor / Marina
170,Downtown Toronto,4,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boutique,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Harbor / Marina
171,Downtown Toronto,4,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boutique,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Harbor / Marina
172,Downtown Toronto,4,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boutique,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Harbor / Marina
173,Downtown Toronto,4,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boutique,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Harbor / Marina


## Clusters Analysis Observations
From the clusters above
* <b>Cluster1:</b> Is in Central Toronto with top 10 common venues frequecies aligning with every venue in this cluster, Coffee Shop, Sushi, American... 
* <b>Cluster2:</b> Indicated in Purple on the Map, it is spread out everywhere through East, Downtown , West Toronto with all of the dissimilar items clustered together.  k-means doesn't have similarity information between venues so it has clutered all of the venues which doesn't fall into any other clusters have grouped together to form this cluster.
* <b>Cluster3:</b> There is only single record in Cluster2 with Garden, although we know that Garden is very similar to Park  in Cluster1 but the k-means doesn't know the similarities in venues. That is why it categorized as just 1 item in cluster.
* <b>Cluster4:</b> All the Parks in Central and Downtown Toronto have grouped together to form this cluster.
* <b>Cluster5:</b> All Airport venues in Downtown Toronto have grouped to form this cluster